In [ ]:
!pip install psycopg2-binary
!pip install sqlalchemy
!pip install xgboost
!pip install mlxtend
!pip install imblearn

In [4]:
import pandas as pd
import psycopg2
import sqlalchemy 
import pyodbc
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, f1_score
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from collections import Counter
from mlxtend.plotting import plot_decision_regions

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler





     -------------------------------------- 235.6/235.6 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 302.2/302.2 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [34]:
params = {
    "host": "localhost",
    "user": "postgres",
    "port": 5432,
    "password": "Aibu2008" 
}
connection = psycopg2.connect(**params, dbname= "postgres")

In [35]:
df=pd.read_sql('select * from public.customer_churn',connection)

In [36]:
connection.close()

In [37]:
df

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [38]:
df.drop(columns='customer_id',inplace=True) #customer id anlamlı bir feature olmadığından siliyoruz

In [39]:
df.info() 
"""
hiç null değer yok. Object olan country ve gender için get dummies yapalım
age değişkeni için de Discretization uygulayalım.
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      10000 non-null  int64  
 1   country           10000 non-null  object 
 2   gender            10000 non-null  object 
 3   age               10000 non-null  int64  
 4   tenure            10000 non-null  int64  
 5   balance           10000 non-null  float64
 6   products_number   10000 non-null  int64  
 7   credit_card       10000 non-null  int64  
 8   active_member     10000 non-null  int64  
 9   estimated_salary  10000 non-null  float64
 10  churn             10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


'\nhiç null değer yok. Object olan country ve gender için get dummies yapalım\nage değişkeni için de Discretization uygulayalım.\n'

In [40]:
df = pd.get_dummies(df, columns=['country','gender'],  drop_first=True)
df

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1


In [44]:
df.describe() #yaş min 18 max 92 olduğunu gördük. 18-32 / 33-44 / 45-60 / 60+ olarak gruplayabiliriz

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.250900,0.247700,0.545700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.433553,0.431698,0.497932
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,0.000000,1.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,0.000000,1.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


In [46]:
def discretizate(age):
    if age <=32:
        return '18-32'
    elif age <=44:
        return '33-44'
    elif age <=60:
        return '45-60'
    else:
        return '60+'

In [47]:
df['age_group'] = df['age'].apply(discretizate)
df

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male,age_group
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0,33-44
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0,33-44
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0,33-44
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0,33-44
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0,33-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1,33-44
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1,33-44
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0,33-44
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1,33-44


In [48]:
age_group_dic = {'60+':3, '45-60':2, '33-44':1, '18-32':0}  

df['age_group'] = df['age_group'].map(age_group_dic)
df

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male,age_group
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0,1
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,0,1,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,0,1,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,0,1
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,0,1,1


In [49]:
df.drop(columns='age',inplace=True) #age kolonunu da drop edelim.

In [53]:
df.info() #tüm değişknlerin tipi int ya da float oldu
df.isnull().sum() #hiç null değer de yok

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      10000 non-null  int64  
 1   tenure            10000 non-null  int64  
 2   balance           10000 non-null  float64
 3   products_number   10000 non-null  int64  
 4   credit_card       10000 non-null  int64  
 5   active_member     10000 non-null  int64  
 6   estimated_salary  10000 non-null  float64
 7   churn             10000 non-null  int64  
 8   country_Germany   10000 non-null  uint8  
 9   country_Spain     10000 non-null  uint8  
 10  gender_Male       10000 non-null  uint8  
 11  age_group         10000 non-null  int64  
dtypes: float64(2), int64(7), uint8(3)
memory usage: 732.5 KB


credit_score        0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
country_Germany     0
country_Spain       0
gender_Male         0
age_group           0
dtype: int64

In [56]:
df

,credit_score,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male,age_group
0,619,2,0.00,1,1,1,101348.88,1,0,0,0,1
1,608,1,83807.86,1,0,1,112542.58,0,0,1,0,1
2,502,8,159660.80,3,1,0,113931.57,1,0,0,0,1
3,699,1,0.00,2,0,0,93826.63,0,0,0,0,1
4,850,2,125510.82,1,1,1,79084.10,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,5,0.00,2,1,0,96270.64,0,0,0,1,1
9996,516,10,57369.61,1,1,1,101699.77,0,0,0,1,1
9997,709,7,0.00,1,0,1,42085.58,1,0,0,0,1
9998,772,3,75075.31,2,1,0,92888.52,1,1,0,1,1


In [ ]:
#train ve test ayrımı ile modelleri deneyelim.